<a href="https://colab.research.google.com/github/Rizukaf-id/StyleMate-ML/blob/main/capstone%20(1).ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
pip install split-folders

In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [2]:
import zipfile
# import splitfolders
import os
import tensorflow as tf

In [3]:
# Extract the archive
local_zip = '/content/drive/MyDrive/dataset_capstone/dataset_capstone/archive_2.zip'
zip_ref = zipfile.ZipFile(local_zip, 'r')
zip_ref.extractall('data/')
zip_ref.close()

In [16]:
# delete class
!rm -rf '/content/data/FaceShape Dataset/testing_set/Oval'
!rm -rf '/content/data/FaceShape Dataset/testing_set/Oblong'
!rm -rf '/content/data/FaceShape Dataset/training_set/Oval'
!rm -rf '/content/data/FaceShape Dataset/training_set/Oblong'

In [ ]:
# base_dir = 'data/dataset_capstone/'
# splitfolders.ratio(base_dir, output = 'data/dataset_capstone/dataset', seed = 1337, ratio=(0.8, 0.2), group_prefix=None)
# train_dir = 'data/dataset_capstone/dataset/train'
# val_dir = 'data/dataset_capstone/dataset/val'

In [17]:
train_dir = '/content/data/FaceShape Dataset/training_set'
val_dir = '/content/data/FaceShape Dataset/testing_set'

In [34]:
from tensorflow.keras.preprocessing.image import ImageDataGenerator

training_datagen = ImageDataGenerator(
      rescale = 1./255,
      rotation_range=30,
      width_shift_range=0.2,
      height_shift_range=0.2,
      shear_range=0.2,
      zoom_range=0.2,
      horizontal_flip=True,
      fill_mode='constant')

validation_datagen = ImageDataGenerator(
                    rescale=1./255)

In [35]:
train_generator = training_datagen.flow_from_directory(
	train_dir,
	target_size=(224,224),
	class_mode='categorical',
  batch_size=32
)

validation_generator = validation_datagen.flow_from_directory(
	val_dir,
	target_size=(224,224),
	class_mode='categorical',
  batch_size=32
)

Found 2400 images belonging to 3 classes.
Found 600 images belonging to 3 classes.


In [37]:
model = tf.keras.models.Sequential([
    tf.keras.layers.Conv2D(64, (3,3), activation='relu', input_shape=(224, 224, 3)),
    tf.keras.layers.MaxPooling2D(2, 2),

    tf.keras.layers.Conv2D(64, (3,3), activation='relu'),
    tf.keras.layers.MaxPooling2D(2,2),

    tf.keras.layers.Conv2D(128, (3,3), activation='relu'),
    tf.keras.layers.MaxPooling2D(2,2),

    tf.keras.layers.Conv2D(256, (3,3), activation='relu'),
    tf.keras.layers.MaxPooling2D(2,2),

    tf.keras.layers.Flatten(),
    tf.keras.layers.Dropout(0.5),

    tf.keras.layers.Dense(512, activation='relu'),
    tf.keras.layers.Dense(3, activation='softmax')
])

In [28]:
# model.add(tf.keras.layers.BatchNormalization())

In [38]:
model.summary()

Model: "sequential_4"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv2d_16 (Conv2D)          (None, 222, 222, 64)      1792      
                                                                 
 max_pooling2d_16 (MaxPooli  (None, 111, 111, 64)      0         
 ng2D)                                                           
                                                                 
 conv2d_17 (Conv2D)          (None, 109, 109, 64)      36928     
                                                                 
 max_pooling2d_17 (MaxPooli  (None, 54, 54, 64)        0         
 ng2D)                                                           
                                                                 
 conv2d_18 (Conv2D)          (None, 52, 52, 128)       73856     
                                                                 
 max_pooling2d_18 (MaxPooli  (None, 26, 26, 128)      

In [40]:
model.compile(loss = 'categorical_crossentropy', optimizer=tf.keras.optimizers.Adam(learning_rate=0.0001), metrics=['accuracy'])

In [31]:
# model.compile(loss='categorical_crossentropy', optimizer=tf.keras.optimizers.SGD(learning_rate=0.01, momentum=0.9), metrics=['accuracy'])

In [41]:
from PIL import ImageFile
ImageFile.LOAD_TRUNCATED_IMAGES = True

In [42]:
# Train the model
history = model.fit(train_generator, epochs=15, validation_data = validation_generator, verbose = 1)

Epoch 1/15
75/75 [==============================] - 49s 592ms/step - loss: 1.1016 - accuracy: 0.3467 - val_loss: 1.0979 - val_accuracy: 0.3267
Epoch 2/15
75/75 [==============================] - 46s 613ms/step - loss: 1.0966 - accuracy: 0.3442 - val_loss: 1.1005 - val_accuracy: 0.3333
Epoch 3/15
75/75 [==============================] - 48s 637ms/step - loss: 1.0956 - accuracy: 0.3617 - val_loss: 1.0974 - val_accuracy: 0.3350
Epoch 4/15
75/75 [==============================] - 44s 588ms/step - loss: 1.0952 - accuracy: 0.3562 - val_loss: 1.0876 - val_accuracy: 0.3933
Epoch 5/15
75/75 [==============================] - 42s 560ms/step - loss: 1.0833 - accuracy: 0.3988 - val_loss: 1.0659 - val_accuracy: 0.4333
Epoch 6/15
75/75 [==============================] - 44s 586ms/step - loss: 1.0698 - accuracy: 0.4162 - val_loss: 1.0570 - val_accuracy: 0.4300
Epoch 7/15
75/75 [==============================] - 45s 596ms/step - loss: 1.0677 - accuracy: 0.4271 - val_loss: 1.0574 - val_accuracy: 0.4433

In [ ]:
pip install tensorflowjs

In [ ]:
import tensorflowjs

In [ ]:
import time
saved_model_path = "./{}.h5".format(int(time.time()))

model.save(saved_model_path)

In [ ]:
!tensorflowjs_converter --input_format=keras {saved_model_path} ./

In [ ]:
saved_model_path = "./{}.h5".format(int(time.time()))

history.save(saved_model_path)

In [ ]:
!tensorflowjs_converter --input_format=keras {saved_model_path} ./